In [1]:
!pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# !pip install -U profanity-filter[multilingual]
!pip install python-obscene-words-filter

In [3]:
import pandas as pd
import re
import random
from profanity_filter import ProfanityFilter
from obscene_words_filter.default import get_default_filter
import obscene_words_filter
filter_obscene_regex = get_default_filter()


# грузим существующий вид гибридного датасета для будущей очистки от дублей

In [ ]:
df_current_hybrid = pd.read_csv("./current_final_version_of_hybrid_dataset/HYBRID_Steps12_unsafe_only.csv")

In [ ]:
df_current_hybrid.head()

In [ ]:
existing_current_hybrid_samples_set = set(df_current_hybrid['text'])

# подгружаем датасет под будущую разметку и проверяем на предмет предобработки

In [ ]:
df_dict = pd.read_csv("alternative_partition - all_topics.csv", header = None)
df_dict.head()

In [ ]:
rus2en = {}
for i,el in df_dict.iterrows():
    rus2en[el[0]] = el[1]
    

In [ ]:
# df = pd.read_csv("crowd_vs_part1self - crowd_vs_part1self.csv")
# df = pd.read_csv("../xlm/mailVSdvach_automatically_marked.csv")
# df = pd.read_csv("processed_data_for_clarification/OtvetiVS2ch_preprocessed_iteration2_automarked.csv")
# df = pd.read_csv("processed_data_for_clarification/SELF_LABLE_export_ready_st2_multilabel_view.csv")
# df = pd.read_csv("../data/HYBRID_ST123_recalcToxic_recalc_multi.csv")
# df = pd.read_csv("../../../../input/stories_1d5mln.csv")

df = pd.read_csv("FOR_HYBRID4_to_obscene.csv")

In [ ]:
# del df['toxic']
# del df['obscene']

In [ ]:
df.columns = [rus2en[el] if el in rus2en else el for el in df.columns]

In [ ]:
df.head(2)

# 1) СНАЧАЛА ЧИСТИМ ОТ МАТА ДЛЯ УМЕНЬШЕНИЯ НАГРУЗКИ НА МОДЕЛЬ

In [5]:
exclude_list = ['какая','сила','дурак','хрен','дураки','мужу','муж','сосать','сосала',
                'минут','минута','сил','стерва','блин','муд','дот','малафьи','гад','соси','сосать']

In [4]:

mat_initial_form = []
with open("../data_text/MAT_FINAL.txt") as f:
    for word in f.readlines():
        w = word[:-1]
        if w not in exclude_list:
            mat_initial_form.append(w)

NameError: name 'exclude_list' is not defined

In [ ]:
mat_initial_form[:2]

In [ ]:
mat_initial_form.index("заебал")

# СКЛОНЯЕМ ОДИНОЧНЫЕ СЛОВА

In [ ]:
!pip install pymorphy2 --upgrade

In [ ]:
>>> import pymorphy2
>>> morph = pymorphy2.MorphAnalyzer()

In [ ]:
morph.parse('ебашить')[0].lexeme

In [ ]:
# inflected_single_words_list = []
# for mat in mat_initial_form:
#     if len(mat.split()) == 1:
#         inflected_words = [el.word for el in morph.parse(mat)[0].lexeme]
#         inflected_words = list(set(inflected_words))
#         inflected_single_words_list.extend(inflected_words)

In [ ]:
# inflected_single_words_list.index("заебали")

In [ ]:
# random.sample(inflected_single_words_list, 10)

In [ ]:
mat_initial_form.extend(inflected_single_words_list)

In [ ]:
mat_initial_form = list(set(mat_initial_form))

In [ ]:
mat_initial_form = []
with open("../data_text/MAT_FINAL_with_unigram_inflections.txt", 'r', encoding = 'utf-8') as f:
    for m in f.readlines():
        m_cl = re.sub('\n','',m)
        mat_initial_form.append(m_cl)

In [ ]:
# with open("MAT_FINAL_with_unigram_inflections.txt", 'w') as f:
#     for m in mat_initial_form:
#         f.write(m+'\n')

In [ ]:
random.sample(mat_initial_form, 10)

In [ ]:
len(mat_initial_form)

In [ ]:
mat_initial_form.index("ебнутая")

In [ ]:
import spacy
nlp = spacy.load("ru")

In [31]:
!pip install spacy --upgrade

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\N.Babakov\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python38\\site-packages\\~rsly\\msgpack\\_packer.cp38-win_amd64.pyd'
Check the permissions.




  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107102 sha256=4b78802226e4492a6f78878ee9caf3e8e532d4de17460a910e8bc0807f04c2f8
  Stored in directory: c:\users\n.babakov\appdata\local\packages\pythonsoftwarefoundation.python.3.8_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\11\73\9a\f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5


In [30]:
# !python3 -m spacy download en
!python -m spacy download ru_core_news_lg


[x] No compatible model found for 'ru' (spaCy v2.3.5).



In [29]:
import spacy
from profanity_filter import ProfanityFilter

nlp = spacy.load('en')
profanity_filter = ProfanityFilter(nlps={'en': nlp})  # reuse spacy Language (optional)
nlp.add_pipe(profanity_filter.spacy_component, last=True)

doc = nlp('This is shiiit!')

doc._.is_profane
# True

doc[:2]._.is_profane
# False

for token in doc:
    print(f'{token}: '
          f'censored={token._.censored}, '
          f'is_profane={token._.is_profane}, '
          f'original_profane_word={token._.original_profane_word}'
    )
        

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [20]:
t = "Да ебнутая, бля ебать уеба это просто ебнутая какой-то!"
for match in f.find_bad_word_matches_without_good_words(t): 
    start, end = match.span()
    bad_word = t[start: end]
    print(bad_word)

бля
ебать


In [ ]:
pf = ProfanityFilter(languages=['ru'])#

pf.censor("Да бля, это просто shit какой-то!")

# Делим на одиночные слова и словосочетания

In [ ]:
import random

In [ ]:
unigram_mat_set = set()
ngram_mat_set = set()
for mat in mat_initial_form:
    if len(mat.split()) == 1:
        if mat not in exclude_list:
            unigram_mat_set.add(mat)
    else:
        ngram_mat_set.add(mat)

In [ ]:
'сил' in unigram_mat_set


In [ ]:
random.sample(ngram_mat_set, 10)

# ПРОГОНЯЕМ ДАТАСЕТ ЧЕРЕЗ ВЫЯВЛЕНИЕ МАТА

In [ ]:
import re
from collections import Counter

In [23]:
# tsttxt = 'превет.ну их всех в манду'
tsttxt = 'и да, в принципе у нас против Зенита никогда ничего не имели, это москвичи всех бля.'

# tsttxt = re.sub('[^a-zA-Zа-яА-Я]', ' ', tsttxt)
# tsttxt = re.sub(' +', ' ', tsttxt)

In [24]:
any(i>2 for i in range(4))

True

In [25]:
def is_any_obscene(txt):
    #ищем в униграммах
    txt = txt.lower()
    txt = re.sub('[^a-zA-Zа-яА-Я]', ' ', txt)
    txt = re.sub(' +', ' ', txt)
    txt = txt.split()
    
    for match in filter_obscene_regex.find_bad_word_matches_without_good_words(t): 
        start, end = match.span()
        bad_word = t[start: end]
        return bad_word
    
    any_obscene_unigram = any(word in unigram_mat_set for word in txt)
    if any_obscene_unigram:
        for word in txt:
            if word in unigram_mat_set:
                return word
    
    txt_join = ' '.join(txt)
    any_obscene_ngram = any(ngram in txt_join for ngram in ngram_mat_set)
    if any_obscene_ngram:
        for ngram in ngram_mat_set:
            if ngram in txt_join:
                return ngram
    
    return 'clean'
    
is_any_obscene(tsttxt)

'бля'

In [ ]:
df['is_obscene'] = df['text'].apply(is_any_obscene)

In [ ]:
Counter(df['is_obscene']).most_common(100)

In [ ]:
df_cleaned = df[df['is_obscene']=='clean']

In [ ]:
len(df_cleaned), len(df)

In [ ]:
del df_cleaned['is_obscene']

In [ ]:
len(df_cleaned)

In [ ]:
# df_cleaned.to_csv("../../../../input/stories_1d5mln_noObscene.csv", index = None)
df_cleaned.to_csv("FOR_HYBRID4_NO_obscene.csv", index = None)

In [ ]:
df_cleaned = df[df['is_obscene']=='clean']
df_cleaned = df_cleaned[df_cleaned['toxic_new']<0.75]
len(df), len(df_cleaned),len(df_cleaned[df_cleaned['unsafe']>0.75]), len(df_cleaned[df_cleaned['unsafe']>0.75])/len(df_cleaned)

In [ ]:
del df_cleaned['is_obscene']

In [ ]:
df_cleaned.to_csv("../data/HYBRID_ST123_CLEANED_FINAL.csv", index = None)

In [ ]:
df.info()

In [ ]:
counter = Counter(df['is_obscene']).most_common(100)
counter.reverse()
counter

In [ ]:
df.head()

In [ ]:
df = df[df['is_obscene'] == 'clean']

In [ ]:
df.head(2)

# УБИРАЕМ ДУБЛИ С СУЩЕСТВУЮЩИМ ГИБРИДНЫМ ДАТАСЕТОМ

In [ ]:
def is_present_in_existing_hybrid(txt):
    if txt in existing_current_hybrid_samples_set:
        return None
    return False

In [ ]:
tt = "Их много и под руководством Рамзана они не станут ваххабитами."
tt in existing_current_hybrid_samples_set

In [ ]:
df_clean['is_duplicate'] = df_clean['text'].apply(is_present_in_existing_hybrid)

In [ ]:
Counter(df_clean['is_duplicate'])

In [ ]:
df_clean.dropna(inplace = True)

# Экспортируем очищенный от токсика и мата датасет

In [ ]:
# df_clean.to_csv("crowd_vs_part1self_cleaned_for_hybrid.csv", index = None)
# df_clean.to_csv("mailVSdvach_automatically_marked_cleaned_gor_hybrid.csv", index = None)
# df_clean.to_csv("mailVSdvach_automatically_marked_cleaned_for_hybrid_ITER2.csv", index = None)
df_clean.to_csv("Self_labeled_cleaned_for_hybrid.csv", index = None)